# Logit lens

##  Содержание
* Описание метода logit lens
* Применение к модели

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoProcessor, LlavaForConditionalGeneration
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import requests
from io import BytesIO
import random
import os
import random
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

random.seed(42)

## Описание метода logit lens
Logit Lens используется для оценки промежуточных предсказаний трансформера. После каждого слоя берется скрытое состояние (hidden state), к нему применяется финальная нормализация слоя (если она есть), затем выходная линейная голова​, используемая при обычном предсказании токенов. После применения софтмакса получается распределение вероятности слов из словаря.Это позволяет узнать, какие токены были бы выбраны моделью, если бы предсказание делалось на этом этапе. Применение финальной головы позволяет понять насколько на данном этапе модель делает хорошие предсказания, нормализация применяется тк без неё масштаб будет отличаться от выходного слоя и приведет к некорректным результатам.
Этот метод показывает как меняются логиты из более зашумленных на ранних слоях к более точным на поздних. Однако скрытые состояния промежуточных слоёв могут не лежать в том же линейном пространстве, что и выходной слой. Поэтому полученные логиты могут быть неточными, в таких случаях возможна ошибка интерпретации, так как промежуточные слои могут использовать другие представления. Просто алгоритма является и плюсом и минусом, было показано что метод не может сделать предсказания на хорошем уровне для моделей GPT-Neo, BLOOM, OPT 125M, плохо работает с нестандартными блоками.


## Датасеты
Для применения мультимодальных моделей возьмём два датасета с изображениями, различающиеся по сложности и содержанию.
1. CLEVR
2. COCO (Common Objects in Context)

In [3]:
# код загрузки датасета CLEVR
# далее 500 картинок были сохранены для более удобного использования
import kagglehub

path = kagglehub.dataset_download("timoboz/clevr-dataset")

val_dir = os.path.join(path, "CLEVR_v1.0", "images", "val")
all_images = sorted(glob.glob(os.path.join(val_dir, "*.png")))

selected_images = random.sample(all_images, 500)
output_dir = "clevr_val_subset_500"
os.makedirs(output_dir, exist_ok=True)
for src_path in selected_images:
    filename = os.path.basename(src_path)
    shutil.copy(src_path, os.path.join(output_dir, filename))
shutil.make_archive("clevr_val_subset_500", "zip", "clevr_val_subset_500") # если нужно сорхранить локально

In [ ]:
# датасет COCO был скачен по ссылке https://cocodataset.org/#download версия val2017
import zipfile
import os
import random
import shutil

zip_path = "val2017.zip"
extracted_dir = "val2017_full/val2017"
subset_dir = "coco_val_500"

os.makedirs(extracted_dir, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)
image_files = [f for f in os.listdir(extracted_dir) if f.endswith(".jpg")]
random.seed(42)
selected = random.sample(image_files, 500)

os.makedirs(subset_dir, exist_ok=True)
for fname in selected:
    shutil.copy(os.path.join(extracted_dir, fname), os.path.join(subset_dir, fname))
shutil.make_archive("coco_val_500", "zip", "coco_val_500") # если нужно сорхранить локально

In [4]:
# при загрузке из зип файлов
import zipfile

def unzip_file(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

unzip_file("clevr_val_subset_500.zip", "clevr_dataset")
unzip_file("coco_val_500.zip", "coco_dataset")

## Модели
Для анализа было выбрано 3 модели:
1. llava-onevision на 1.5b
2. qwen2-vl на 2b

Они имеют схожую структуру:
* Визуальный модуль- разбивает изображение на патчи, добавляет позиционный энкодинг, преобразует изображение в эмбеддинги слоями трансформера (CLIPVisionTransformer для LLaVA-OneVision, VisionTransformer для Qwen2-VL)

* Мультимодальная проекция - приводит эмбеддинги изображений к нужной текстовой размерности

* Языковая модель, блок который будет использоваться с помощью метода logit lens
  * LLaMa (0-31)
  * Qwen2-2B (0-23)



## Исследование динамики модели
Для каждого датасета возьмём 5 картинок и визуально посмотрим на изменение токенов на каждом датасете и уверенность модели


### llava-onevision на 1.5b

In [4]:
# Загрузка модели
model_id = "llava-hf/llava-1.5-7b-hf"
model_llava = LlavaForConditionalGeneration.from_pretrained(
  model_id,
  torch_dtype=torch.float16,
  low_cpu_mem_usage=True,
).to(0)
processor = AutoProcessor.from_pretrained(model_id)

In [8]:
def get_hidden_states(prompt, img, model):
    # Обработка текстового запроса
    inputs = processor.apply_chat_template(prompt, add_generation_prompt=True)
    inputs = processor(text=inputs, images=img, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # сохранение логитов на каждом шаге
    with torch.no_grad():
        outputs = model(
            **inputs,
            output_hidden_states=True,
            return_dict=True
        )
        hidden_states = outputs.hidden_states
    return hidden_states, inputs["input_ids"].shape[1]

# При визуализации нескольких топ токенов возникала проблема, что split() превращал
# специальные токены в пустые строки
def safe_decode(token_id: int, tokenizer) -> str:

    raw = tokenizer.decode([token_id], skip_special_tokens=False)

    if token_id in tokenizer.all_special_ids or raw in tokenizer.all_special_tokens:
        return f"<{raw.strip() or 'SPECIAL'}>"

    if raw.strip() == "":
        if raw == " ":
            return "<SPACE>"
        elif raw == "\n":
            return "<NEWLINE>"
        elif raw == "":
            return "<EMPTY>"

    return raw

def get_visuals(hidden_states, img, model, len_inputs):
    # подготовим данные к визуализации
    # уверенность модели в первом токене
    # топ-3 токенов
    target_pos = len_inputs - 1

    layer_confidences = []
    top_tokens = []
    seq_len = hidden_states[-1].shape[1]
    target_positions = list(range(seq_len - 3, seq_len))
    for i, hs in enumerate(hidden_states):
        logits =  model.lm_head(model.language_model.norm(hs))
        max_probs = torch.softmax(logits, dim=-1).max(dim=-1).values
        layer_confidences.append(max_probs.mean().item())

        topk = torch.topk(logits, k=3, dim=-1)
        top_ids = topk.indices[0, target_pos]
        top_strs = [safe_decode(i.item(), processor.tokenizer) for i in top_ids]
        top_tokens.append(top_strs)
        del logits
        del max_probs

    # визуализация
    plt.figure(figsize=(10, 4))
    sns.lineplot(x=list(range(len(layer_confidences))), y=layer_confidences, marker="o")
    plt.xlabel("Layer index")
    plt.ylabel("Average max softmax probability")
    plt.title("Logit Lens Confidence Across Layers")
    plt.grid(True)
    plt.show()

    fig, ax = plt.subplots(figsize=(12, 6))
    table_data = [
        [f"Layer {i}", t[0], t[1], t[2]] for i, t in enumerate(top_tokens)
    ]
    col_labels = ["Layer", "Top-1", "Top-2", "Top-3"]

    table = ax.table(
        cellText=table_data,
        colLabels=col_labels,
        loc='center',
        cellLoc='left'
    )
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.1, 1.2)
    ax.axis('off')
    plt.tight_layout()
    plt.show()

In [2]:
seed = 42
random.seed(seed)
torch.manual_seed(seed)
clevr_folder = "/content/clevr_dataset"
coco_folder = "/content/coco_dataset"

clevr_paths = random.sample(
    [os.path.join(clevr_folder, f) for f in os.listdir(clevr_folder) if f.endswith(".png")], 5)

coco_paths = random.sample(
    [os.path.join(coco_folder, f) for f in os.listdir(coco_folder) if f.endswith(".jpg")], 5)

clevr_images = [Image.open(p).convert("RGB") for p in clevr_paths]
coco_images = [Image.open(p).convert("RGB") for p in coco_paths]

In [1]:

clevr_prompts = [
    "Color of the main object:",
    "Shape of the main object:",
    "Material of the main object:",
    "Number of main objects:",
]

coco_prompts = [
    "Main object:",
    "Answer with one word. What is the person doing:",
    "Place:",
    "Main animal:",
    "Answer with one word. What is shown in the background:"
]



for dataset_name, image_list, prompts in [("CLEVR", clevr_images, clevr_prompts), ("COCO", coco_images, coco_prompts)]:
    for i, img in enumerate(image_list):
        print(f"\n===== {dataset_name} Image {i+1} =====")
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"{dataset_name} Image {i+1}")
        plt.show()

        for j, prompt_text in enumerate(prompts):
            print(f"\n--- Prompt {j+1}: {prompt_text} ---")
            prompt = [{"role": "user", "content": [
                {"type": "image", "data": img},
                {"type": "text", "text": prompt_text}
            ]}]
            hidden_states, len_inputs = get_hidden_states(prompt, img, model_llava)
            get_visuals(hidden_states, img, model_llava, len_inputs)
            del hidden_states

In [11]:
# Эксперименты с изменёнными запросами
clevr_prompts = [
    "Color of the smaller object:",
    "Answer using one word. 3D Shape of the main object:",
    "Answer using one word. Shape of the object to the left of the cube:",
]

for dataset_name, image_list, prompts in [("CLEVR", clevr_images, clevr_prompts), ("COCO", coco_images, coco_prompts)]:
    for i, img in enumerate(image_list):
        print(f"\n===== {dataset_name} Image {i+1} =====")
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"{dataset_name} Image {i+1}")
        plt.show()

        for j, prompt_text in enumerate(prompts):
            print(f"\n--- Prompt {j+1}: {prompt_text} ---")
            prompt = [{"role": "user", "content": [
                {"type": "image", "data": img},
                {"type": "text", "text": prompt_text}
            ]}]
            hidden_states, len_inputs = get_hidden_states(prompt, img, model_llava)
            get_visuals(hidden_states, img, model_llava, len_inputs)
            del hidden_states

In [11]:
model_llava

LlavaForConditionalGeneration(
  (model): LlavaModel(
    (vision_tower): CLIPVisionModel(
      (vision_model): CLIPVisionTransformer(
        (embeddings): CLIPVisionEmbeddings(
          (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
          (position_embedding): Embedding(577, 1024)
        )
        (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (encoder): CLIPEncoder(
          (layers): ModuleList(
            (0-23): 24 x CLIPEncoderLayer(
              (self_attn): CLIPAttention(
                (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
              )
              (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
       

# Qwen

In [5]:
model_qwen = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct", torch_dtype="auto", device_map="auto"
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

In [2]:
clevr_prompts = [
    "Color of the main object:",
    "Shape of the main object:",
    "Material of the main object:",
    "Number of main objects:",
]

coco_prompts = [
    "Main object:",
    "Answer with one word. What is the person doing:",
    "Place:",
    "Main animal:",
    "Answer with one word. What is shown in the background:"
]

for dataset_name, image_list, prompts in [("CLEVR", clevr_images, clevr_prompts), ("COCO", coco_images, coco_prompts)]:
    for i, img in enumerate(image_list):
        print(f"\n===== {dataset_name} Image {i+1} =====")
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"{dataset_name} Image {i+1}")
        plt.show()

        for j, prompt_text in enumerate(prompts):
            print(f"\n--- Prompt {j+1}: {prompt_text} ---")
            prompt = [{"role": "user", "content": [
                {"type": "image", "data": img},
                {"type": "text", "text": prompt_text}
            ]}]
            hidden_states, len_inputs = get_hidden_states(prompt, img, model_qwen)
            get_visuals(hidden_states, img, model_qwen, len_inputs)
            del hidden_states

In [ ]:
model_qwen

Qwen2VLForConditionalGeneration(
  (model): Qwen2VLModel(
    (visual): Qwen2VisionTransformerPretrainedModel(
      (patch_embed): PatchEmbed(
        (proj): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
      )
      (rotary_pos_emb): VisionRotaryEmbedding()
      (blocks): ModuleList(
        (0-31): 32 x Qwen2VLVisionBlock(
          (norm1): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
          (norm2): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
          (attn): VisionAttention(
            (qkv): Linear(in_features=1280, out_features=3840, bias=True)
            (proj): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (mlp): VisionMlp(
            (fc1): Linear(in_features=1280, out_features=5120, bias=True)
            (act): QuickGELUActivation()
            (fc2): Linear(in_features=5120, out_features=1280, bias=True)
          )
        )
      )
      (merger): PatchMerger(
        (ln_q): LayerN